# Download Images and Libraries

In [ ]:
!mkdir Images
!mkdir Output_Images
!mkdir Output_Images/Convolution
!mkdir Output_Images/Pooling

!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
!wget https://raw.githubusercontent.com/frischzenger/yolo-windows/master/3rdparty/include/pthread.h

!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/9cb838f170745b866d13da5476affb7f51425e33/img0.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/9cb838f170745b866d13da5476affb7f51425e33/img1.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img2.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img3.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img4.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img5.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img6.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img7.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img8.png
!wget -P /content/Images https://raw.githubusercontent.com/NathanBruggeman/Images_Cuda/main/img9.png

--2023-01-08 12:46:47--  https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279339 (273K) [text/plain]
Saving to: ‘stb_image.h’

stb_image.h         100%[===================>] 272.79K  --.-KB/s    in 0.004s  

2023-01-08 12:46:47 (73.0 MB/s) - ‘stb_image.h’ saved [279339/279339]

--2023-01-08 12:46:47--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_

# Write and compile the ImageFilter_Sequential.c
## This program filters 10 images sequential.

In [ ]:
%%writefile ImageFilter_Sequential.c
#include <stdio.h>
#include <stdlib.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image.h"
#include "stb_image_write.h"
#include <string.h>
#include <time.h>

typedef struct Pixel
{
    unsigned char r, g, b, a;
} Pixel;

void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageDataGrayscale, int width, int height);
void convolveImage(unsigned char *imageDataGrayscale, unsigned char *imageDataConvolution, int width, int height);
void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height);
void maxPooling(unsigned char *originalImage, unsigned char *maxPoolingImage, int width, int height);

int main()
{
    clock_t timer_start, timer_end;
    timer_start = clock(); // Start the timer

    for (int i = 0; i < 10; i++)
    {
        char inputImage[64] = "Images/img";
        char outputImageConvolution[64] = "Output_Images/Convolution/img";
        char outputImagePoolingMax[64] = "Output_Images/Pooling/imgMax";
        char outputImagePoolingMin[64] = "Output_Images/Pooling/imgMin";
        char imageNumber[3];

        // Make a complete path for the input (inputImage + imageNumber + extention)
        sprintf(imageNumber, "%d", i); // Get the number of the current iteration and convert it to char imageNumber
        strcat(inputImage, imageNumber);
        strcat(inputImage, ".png");

        ////////////////
        // Open Image //
        ////////////////
        // Open the image and allocate memory for each process
        int width, height, componentCount;
        printf("Loading %s.\r\n", inputImage);
        unsigned char *originalImage = stbi_load(inputImage, &width, &height, &componentCount, 4); // Saves original image
        if (!originalImage)
        {
            printf("Failed to open image!\r\n");
            stbi_image_free(originalImage);
            return -1;
        }

        if (width % 32 || height % 32)
        {
            // NOTE: Leaked memory of "imageData"
            printf("Width and/or Height is not dividable by 32!\r\n");
            stbi_image_free(originalImage);
            return -1;
        }

        unsigned char *imageData = (unsigned char *)malloc(width * height * 4);           // Saves grayscale image
        unsigned char *imageDataTest = (unsigned char *)malloc(width * height * 4);       // Saves output image
        unsigned char *imageDataMinPooling = (unsigned char *)malloc(width * height * 4); // Saves Min pooling image
        unsigned char *imageDataMaxPooling = (unsigned char *)malloc(width * height * 4); // Saves Max pooling image

        ///////////////////////
        // Convert Grayscale //
        ///////////////////////
        // Convert image to grayscale on CPU
        printf("Processing image grayscale.\r\n");
        // Process image on cpu
        ConvertImageToGrayCpu(originalImage, imageData, width, height);
        printf("Done\r\n");

        ////////////////////
        // Convolve image //
        ////////////////////
        // Make a complete path for the convolved image (outputImage convolution + imageNumber + extention)
        strcat(outputImageConvolution, imageNumber);
        strcat(outputImageConvolution, ".png");

        // Convolve the image on CPU
        printf("Convolving image.\r\n");
        convolveImage(imageData, imageDataTest, width, height);
        printf("Done\r\n");

        // Write convolved image to disk
        printf("Writing image to disk\r\n");
        stbi_write_png(outputImageConvolution, width, height, 4, imageDataTest, 4 * width);
        printf("Done\r\n");

        /////////////////
        // Min Pooling //
        /////////////////
        // Make a complete path for max pooling
        sprintf(imageNumber, "%d", i); // Get the number of the current iteration and convert it to char imageNumber
        strcat(outputImagePoolingMin, imageNumber);
        strcat(outputImagePoolingMin, ".png");
        minPooling(originalImage, imageDataMinPooling, width, height);
        // Write image back to disk
        stbi_write_png(outputImagePoolingMin, width / 2, height / 2, 4, imageDataMinPooling, 4 * (width / 2));

        /////////////////
        // Max Pooling //
        /////////////////
        sprintf(imageNumber, "%d", i); // Get the number of the current iteration and convert it to char imageNumber
        strcat(outputImagePoolingMax, imageNumber);
        strcat(outputImagePoolingMax, ".png");
        maxPooling(originalImage, imageDataMaxPooling, width, height);

        // Write image back to disk
        stbi_write_png(outputImagePoolingMax, width / 2, height / 2, 4, imageDataMaxPooling, 4 * (width / 2));

        /////////////////
        // Free memory //
        /////////////////
        stbi_image_free(originalImage);
        free(imageData);
        free(imageDataTest);
        free(imageDataMinPooling);
        free(imageDataMaxPooling);

        printf("\r\nNext Image\r\n");
    }

    ///////////////
    // End timer //
    ///////////////
    timer_end = clock(); // end the timer
    double time_spent = (double)(timer_end - timer_start) / CLOCKS_PER_SEC;
    printf("\nProgram time: %.3fs\n", time_spent);

    return 0;
}

void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageDataGrayscale, int width, int height)
{
    for (int y = 0; y < height; y++)
    {
        for (int x = 0; x < width; x++)
        {
            Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[y * width * 4 + 4 * x];
            Pixel *ptrPixelOriginal = (Pixel *)&originalImage[y * width * 4 + 4 * x];
            unsigned char pixelValue = (unsigned char)(ptrPixelOriginal->r * 0.2126f + ptrPixelOriginal->g * 0.7152f + ptrPixelOriginal->b * 0.0722f);
            ptrPixel->r = pixelValue;
            ptrPixel->g = pixelValue;
            ptrPixel->b = pixelValue;
            ptrPixel->a = 255;
        }
    }
}

void convolveImage(unsigned char *imageDataGrayscale, unsigned char *imageDataConvolution, int width, int height)
{
    int kernel[3][3] =
        {
            {1, 0, -1},
            {1, 0, -1},
            {1, 0, -1}};

    int pixels[3][3] = {0};
    int finalPixel = 0;

    for (int y = 0; y < height - 2; y++)
    {
        for (int x = 0; x < width - 2; x++)
        {
            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[(y * width * 4 + 4 * x) + i * 4];

                pixels[0][i] = ptrPixel->r * kernel[0][i];
            }

            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[(y * width * 4 + 4 * x) + width * 4 + i * 4];

                pixels[1][i] = ptrPixel->r * kernel[1][i];
            }

            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[(y * width * 4 + 4 * x) + (2 * width * 4) + i * 4];

                pixels[2][i] = ptrPixel->r * kernel[2][i];
            }

            finalPixel = (pixels[0][0] + pixels[0][1] + pixels[0][2] + pixels[1][0] + pixels[1][1] + pixels[1][2] + pixels[2][0] + pixels[2][1] + pixels[2][2]) / 9;

            Pixel *ptrPixel = (Pixel *)&imageDataConvolution[(y * width * 4 + 4 * x)];
            ptrPixel->r = finalPixel;
            ptrPixel->g = finalPixel;
            ptrPixel->b = finalPixel;
            ptrPixel->a = 255;
        }
    }
}

void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height)
{
    int counter = 0;

    // Iterate over the image in 2x2 blocks
    for (int y = 0; y < height; y += 2)
    {
        for (int x = 0; x < width; x += 2)
        {
            // For each channel, find the maximum value in the 2x2 block
            for (int c = 0; c < 4; c++)
            {
                Pixel *ptrPixelMinPooling = (Pixel *)&minPoolingImage[counter];
                unsigned char min = 255;
                for (int dy = 0; dy < 2; dy++)
                {
                    for (int dx = 0; dx < 2; dx++)
                    {
                        // Calculate the index of the current pixel in the 1D array
                        int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                        unsigned char value = originalImage[index];
                        min = (value < min) ? value : min;
                    }
                }
                // Store the minimum value in the result array
                ptrPixelMinPooling->r = min;
                ptrPixelMinPooling->g = min;
                ptrPixelMinPooling->b = min;
                ptrPixelMinPooling->a = min;
                counter++;
            }
        }
    }
}

void maxPooling(unsigned char *originalImage, unsigned char *maxPoolingImage, int width, int height)
{
    int counter = 0;

    // Iterate over the image in 2x2 blocks
    for (int y = 0; y < height; y += 2)
    {
        for (int x = 0; x < width; x += 2)
        {
            // For each channel, find the maximum value in the 2x2 block
            for (int c = 0; c < 4; c++)
            {
                Pixel *ptrPixelMaxPooling = (Pixel *)&maxPoolingImage[counter];
                unsigned char max = 0;
                for (int dy = 0; dy < 2; dy++)
                {
                    for (int dx = 0; dx < 2; dx++)
                    {
                        // Calculate the index of the current pixel in the 1D array
                        int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                        unsigned char value = originalImage[index];
                        max = (value > max) ? value : max;
                    }
                }
                // Store the maximum value in the result array
                ptrPixelMaxPooling->r = max;
                ptrPixelMaxPooling->g = max;
                ptrPixelMaxPooling->b = max;
                ptrPixelMaxPooling->a = max;
                counter++;
            }
        }
    }
}




Overwriting ImageFilter_Sequential.c


In [ ]:
!g++ ImageFilter_Sequential.c -o ImageFilter_Sequential

In [ ]:
!./ImageFilter_Sequential

Loading Images/img0.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img1.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img2.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img3.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img4.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img5.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img6.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img7.png.
Processing image grayscale.
Done
Convolving image.
Done
Writing image to disk
Done

Next Image
Loading Images/img8.png.
Processing imag

# Write and compile the ImageFilter_Threads.c
## This program filters 10 images using threads.

In [ ]:
%%writefile ImageFilter_Threads.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image.h"
#include "stb_image_write.h"

// The amount of pictures you have.
// Relies on the fact you name your images correct (ie img11.png).
#define NUM_THREADS 9

// Used to store which thread we are in.
int threadNumber = 0;

typedef struct Pixel
{
    unsigned char r, g, b, a;
} Pixel;

// Function to convert the image from colour to grayscale.
void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageRGBA, int width, int height);
void convolveImage(unsigned char *imageRGBA, unsigned char *imageTest, int width, int height);
void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height);
void maxPooling(unsigned char *originalImage, unsigned char *maxPoolingImage, int width, int height);

// This function runs all the threads
void *runThreads(void *vargp)
{
    clock_t timer_start, timer_end;
    timer_start = clock(); // Start the timer
    // Get which thread we are in, and do ++ for the next thread.
    int threadId = threadNumber++;

    char inputImage[64] = "Images/img";
    char outputImageConvolution[64] = "Output_Images/Convolution/img";
    char outputImagePoolingMax[64] = "Output_Images/Pooling/imgMax";
    char outputImagePoolingMin[64] = "Output_Images/Pooling/imgMin";
    char imageNumber[3];

    // Make a complete path for the input (inputImage + imageNumber + extention)
    sprintf(imageNumber, "%d", threadId); // Get the number of the current iteration and convert it to char imageNumber
    strcat(inputImage, imageNumber);
    strcat(inputImage, ".png");

    // Open image
    int width, height, componentCount;
    unsigned char *originalImage = stbi_load(inputImage, &width, &height, &componentCount, 4); // Saves original image
    if (!originalImage)
    {
        printf("Failed to open Image\r\n");
        exit(-1);
    }

    // Validate image sizes
    if (width % 32 || height % 32)
    {
        // NOTE: Leaked memory of "imageData"
        printf("Width and/or Height is not dividable by 32! ( %s )\r\n", inputImage);
        exit(-1);
    }

    unsigned char *imageData = (unsigned char *)malloc(width * height * 4);           // Saves grayscale image
    unsigned char *imageDataTest = (unsigned char *)malloc(width * height * 4);       // Saves output image
    unsigned char *imageDataMinPooling = (unsigned char *)malloc(width * height * 4); // Saves Min pooling image
    unsigned char *imageDataMaxPooling = (unsigned char *)malloc(width * height * 4); // Saves Max pooling image

    // Process image on cpu
    ConvertImageToGrayCpu(originalImage, imageData, width, height);

    ////////////////////
    // Convolve image //
    ////////////////////
    // Process image on cpu
    convolveImage(imageData, imageDataTest, width, height);

    // Make a complete path for the convolved image (outputImage convolution + imageNumber + extention)
    strcat(outputImageConvolution, imageNumber);
    strcat(outputImageConvolution, ".png");

    // Write image back to disk
    stbi_write_png(outputImageConvolution, width, height, 4, imageDataTest, 4 * width);

    /////////////////
    // Min Pooling //
    /////////////////
    // Make a complete path for max pooling
    sprintf(imageNumber, "%d", threadId); // Get the number of the current iteration and convert it to char imageNumber
    strcat(outputImagePoolingMin, imageNumber);
    strcat(outputImagePoolingMin, ".png");

    minPooling(originalImage, imageDataMinPooling, width, height);

    // Write image back to disk
    stbi_write_png(outputImagePoolingMin, width / 2, height / 2, 4, imageDataMinPooling, 4 * (width / 2));

    /////////////////
    // Max Pooling //
    /////////////////
    sprintf(imageNumber, "%d", threadId); // Get the number of the current iteration and convert it to char imageNumber
    strcat(outputImagePoolingMax, imageNumber);
    strcat(outputImagePoolingMax, ".png");

    maxPooling(originalImage, imageDataMaxPooling, width, height);

    // Write image back to disk
    stbi_write_png(outputImagePoolingMax, width / 2, height / 2, 4, imageDataMaxPooling, 4 * (width / 2));

    stbi_image_free(originalImage);
    free(imageData);
    free(imageDataTest);
    free(imageDataMinPooling);
    free(imageDataMaxPooling);

    timer_end = clock(); // end the timer
    double time_spent = (double)(timer_end - timer_start) / CLOCKS_PER_SEC;
    printf("\n----------\n%s took %.3fs\n----------\n",inputImage, time_spent);
    return NULL;
}

int main()
{
    pthread_t tid;

    // Make NUM_THREADS amount of threads.
    for (int i = 0; i <= NUM_THREADS; i++)
    {
        pthread_create(&tid, NULL, runThreads, NULL);
    }

    pthread_exit(NULL);
    return 0;
}

void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageRGBA, int width, int height)
{
    for (int y = 0; y < height; y++)
    {
        for (int x = 0; x < width; x++)
        {
            Pixel *ptrPixel = (Pixel *)&imageRGBA[y * width * 4 + 4 * x];
            Pixel *ptrPixelOriginal = (Pixel *)&originalImage[y * width * 4 + 4 * x];
            unsigned char pixelValue = (unsigned char)(ptrPixelOriginal->r * 0.2126f + ptrPixelOriginal->g * 0.7152f + ptrPixelOriginal->b * 0.0722f);
            ptrPixel->r = pixelValue;
            ptrPixel->g = pixelValue;
            ptrPixel->b = pixelValue;
            ptrPixel->a = 255;
        }
    }
}

void convolveImage(unsigned char *imageRGBA, unsigned char *imageTest, int width, int height)
{
    int kernel[3][3] =
        {
            {1, 0, -1},
            {1, 0, -1},
            {1, 0, -1}};

    int pixels[3][3] = {0};
    int finalPixel = 0;

    for (int y = 0; y < height - 2; y++)
    {
        for (int x = 0; x < width - 2; x++)
        {
            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageRGBA[(y * width * 4 + 4 * x) + i * 4];

                pixels[0][i] = ptrPixel->r * kernel[0][i];
            }

            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageRGBA[(y * width * 4 + 4 * x) + width * 4 + i * 4];

                pixels[1][i] = ptrPixel->r * kernel[1][i];
            }

            for (int i = 0; i <= 2; i++)
            {
                Pixel *ptrPixel = (Pixel *)&imageRGBA[(y * width * 4 + 4 * x) + (2 * width * 4) + i * 4];

                pixels[2][i] = ptrPixel->r * kernel[2][i];
            }

            finalPixel = (pixels[0][0] + pixels[0][1] + pixels[0][2] + pixels[1][0] + pixels[1][1] + pixels[1][2] + pixels[2][0] + pixels[2][1] + pixels[2][2]) / 9;

            Pixel *ptrPixel = (Pixel *)&imageTest[(y * width * 4 + 4 * x)];
            ptrPixel->r = finalPixel;
            ptrPixel->g = finalPixel;
            ptrPixel->b = finalPixel;
            ptrPixel->a = 255;
        }
    }
}

void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height)
{
    int counter = 0;

    // Iterate over the image in 2x2 blocks
    for (int y = 0; y < height; y += 2)
    {
        for (int x = 0; x < width; x += 2)
        {
            // For each channel, find the maximum value in the 2x2 block
            for (int c = 0; c < 4; c++)
            {
                Pixel *ptrPixelMinPooling = (Pixel *)&minPoolingImage[counter];
                unsigned char min = 255;
                for (int dy = 0; dy < 2; dy++)
                {
                    for (int dx = 0; dx < 2; dx++)
                    {
                        // Calculate the index of the current pixel in the 1D array
                        int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                        unsigned char value = originalImage[index];
                        min = (value < min) ? value : min;
                    }
                }
                // Store the minimum value in the result array
                ptrPixelMinPooling->r = min;
                ptrPixelMinPooling->g = min;
                ptrPixelMinPooling->b = min;
                ptrPixelMinPooling->a = min;
                counter++;
            }
        }
    }
}

void maxPooling(unsigned char *originalImage, unsigned char *maxPoolingImage, int width, int height)
{
    int counter = 0;

    // Iterate over the image in 2x2 blocks
    for (int y = 0; y < height; y += 2)
    {
        for (int x = 0; x < width; x += 2)
        {
            // For each channel, find the maximum value in the 2x2 block
            for (int c = 0; c < 4; c++)
            {
                Pixel *ptrPixelMaxPooling = (Pixel *)&maxPoolingImage[counter];
                unsigned char max = 0;
                for (int dy = 0; dy < 2; dy++)
                {
                    for (int dx = 0; dx < 2; dx++)
                    {
                        // Calculate the index of the current pixel in the 1D array
                        int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                        unsigned char value = originalImage[index];
                        max = (value > max) ? value : max;
                    }
                }
                // Store the maximum value in the result array
                ptrPixelMaxPooling->r = max;
                ptrPixelMaxPooling->g = max;
                ptrPixelMaxPooling->b = max;
                ptrPixelMaxPooling->a = max;
                counter++;
            }
        }
    }
}

Overwriting ImageFilter_Threads.c


In [ ]:
!g++ -pthread ImageFilter_Threads.c -o ImageFilter_Threads

In [ ]:
!./ImageFilter_Threads

# Write and compile the ImageFilter.cu
## This program filters 10 accelerated with CUDA.

Het programma loopt door de grote afbeeldingen tegen de gebruikers limiten van colab aan bij het filteren van 10 afbeeldingen. Om het programma toch te laten werken kan de define worden aangepast naar een maximum van 4 images.

In [ ]:
%%writefile ImageFilter_CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "cuda.h"
#include "cuda_runtime.h"
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image.h"
#include "stb_image_write.h"

#define NUMBER_OF_IMAGES 4 // (Standaard 10 maar door limiten van colab werkt het maar tot 4)

typedef struct Pixel
{
    unsigned char r, g, b, a;
} Pixel;

__global__ void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageDataGrayscale, int width, int height);
__global__ void convolveImage(unsigned char *imageDataGrayscale, unsigned char *imageDataConvolution, int width, int height);
__global__ void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height);
__global__ void maxPooling(unsigned char *originalImage, unsigned char *maxPoolingImage, int width, int height);

int main(int argc, char **argv)
{
    clock_t timer_start, timer_end, timer_start_process, timer_end_process;
    timer_start = clock();

    size_t threadsPerBlock = 128;
    size_t numberOfBlocks = 32;

    cudaStream_t stream[10];

    for (int i = 0; i < 10; i++)
    {
        cudaStreamCreate(&stream[i]);
    }

    printf("Building filepaths\r\n");

    const char *inputFileName[10] =
        {
            "Images/img0.png",
            "Images/img1.png",
            "Images/img2.png",
            "Images/img3.png",
            "Images/img4.png",
            "Images/img5.png",
            "Images/img6.png",
            "Images/img7.png",
            "Images/img8.png",
            "Images/img9.png",
        };

    // Build output filename
    const char *fileNameOutConvolution[10] =
        {
            "Output_Images/Convolution/OutputConvolution0.png",
            "Output_Images/Convolution/OutputConvolution1.png",
            "Output_Images/Convolution/OutputConvolution2.png",
            "Output_Images/Convolution/OutputConvolution3.png",
            "Output_Images/Convolution/OutputConvolution4.png",
            "Output_Images/Convolution/OutputConvolution5.png",
            "Output_Images/Convolution/OutputConvolution6.png",
            "Output_Images/Convolution/OutputConvolution7.png",
            "Output_Images/Convolution/OutputConvolution8.png",
            "Output_Images/Convolution/OutputConvolution9.png",
        };

    const char *fileNameOutMinPooling[10] =
        {
            "Output_Images/Pooling/OutputMinPooling0.png",
            "Output_Images/Pooling/OutputMinPooling1.png",
            "Output_Images/Pooling/OutputMinPooling2.png",
            "Output_Images/Pooling/OutputMinPooling3.png",
            "Output_Images/Pooling/OutputMinPooling4.png",
            "Output_Images/Pooling/OutputMinPooling5.png",
            "Output_Images/Pooling/OutputMinPooling6.png",
            "Output_Images/Pooling/OutputMinPooling7.png",
            "Output_Images/Pooling/OutputMinPooling8.png",
            "Output_Images/Pooling/OutputMinPooling9.png",
        };

    const char *fileNameOutMaxPooling[10] =
        {
            "Output_Images/Pooling/OutputMaxPooling0.png",
            "Output_Images/Pooling/OutputMaxPooling1.png",
            "Output_Images/Pooling/OutputMaxPooling2.png",
            "Output_Images/Pooling/OutputMaxPooling3.png",
            "Output_Images/Pooling/OutputMaxPooling4.png",
            "Output_Images/Pooling/OutputMaxPooling5.png",
            "Output_Images/Pooling/OutputMaxPooling6.png",
            "Output_Images/Pooling/OutputMaxPooling7.png",
            "Output_Images/Pooling/OutputMaxPooling8.png",
            "Output_Images/Pooling/OutputMaxPooling9.png",
        };

    int width[10], height[10], componentCount[10], size[10];

    unsigned char *originalImageHost[10];
    unsigned char *imageDataConvolutionHost[10]; // Saves output image
    unsigned char *imageDataMinPoolingHost[10];  // Saves Min pooling image
    unsigned char *imageDataMaxPoolingHost[10];  // Saves Max pooling image
    unsigned char *originalImage[10];            // Saves the original image on host
    unsigned char *imageDataGrayscale[10];       // Saves the grayscale image on device
    unsigned char *imageDataConvolution[10];     // Saves the convolved image
    unsigned char *imageDataMinPooling[10];      // Saves the min pooled image
    unsigned char *imageDataMaxPooling[10];      // Saves the max pooled image

    printf("Done\r\n");

    printf("Loading png files\r\n");

    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        originalImageHost[i] = stbi_load(inputFileName[i], &width[i], &height[i], &componentCount[i], 4);

        size[i] = height[i] * width[i] * 4;

        // Saves output image
        imageDataConvolutionHost[i] = (unsigned char *)malloc(size[i]);

        // Saves Min pooling image
        imageDataMinPoolingHost[i] = (unsigned char *)malloc(size[i]);

        // Saves Max pooling image
        imageDataMaxPoolingHost[i] = (unsigned char *)malloc(size[i]);

        cudaMalloc(&originalImage[i], size[i]);
        cudaMalloc(&imageDataGrayscale[i], size[i]);
        cudaMalloc(&imageDataConvolution[i], size[i]);
        cudaMalloc(&imageDataMinPooling[i], size[i]);
        cudaMalloc(&imageDataMaxPooling[i], size[i]);

        cudaMemcpy(originalImage[i], originalImageHost[i], size[i], cudaMemcpyHostToDevice);
    }

    printf("Done\r\n");

    timer_start_process = clock();

    // Process grayscale
    printf("Processing images grayscale\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        ConvertImageToGrayCpu<<<numberOfBlocks, threadsPerBlock, i, stream[i]>>>(originalImage[i], imageDataGrayscale[i], width[i], height[i]);
    }
    cudaDeviceSynchronize();
    printf("Done\r\n");

    // Process convolution
    printf("Processing image convolution\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        convolveImage<<<numberOfBlocks, threadsPerBlock, i, stream[i]>>>(imageDataGrayscale[i], imageDataConvolution[i], width[i], height[i]);
    }
    cudaDeviceSynchronize();
    printf("Done\r\n");

    // Process min pooling
    printf("Processing images minimum pooling\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        dim3 gridDim(width[i] / 2, height[i] / 2);
        dim3 blockDim(2, 2);
        // minPooling<<<numberOfBlocks, threadsPerBlock, i, stream[i]>>>(originalImage[i], imageDataMinPooling[i], width[i], height[i]);
        minPooling<<<gridDim, blockDim, i, stream[i]>>>(originalImage[i], imageDataMinPooling[i], width[i], height[i]);
    }
    cudaDeviceSynchronize();
    printf("Done\r\n");

    // Process max pooling
    printf("Processing image maximum pooling\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        dim3 gridDim(width[i] / 2, height[i] / 2);
        dim3 blockDim(2, 2);
        // maxPooling<<<numberOfBlocks, threadsPerBlock, i, stream[i]>>>(originalImage[i], imageDataMaxPooling[i], width[i], height[i]);
        maxPooling<<<gridDim, blockDim, i, stream[i]>>>(originalImage[i], imageDataMaxPooling[i], width[i], height[i]);
    }
    cudaDeviceSynchronize();
    printf("Done\r\n");

    timer_end_process = clock();

    // Writing Convolved images
    printf("Writing convolved png to disk\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        printf("Image %d of %d\r\n", i + 1, 10);
        cudaMemcpy(imageDataConvolutionHost[i], imageDataConvolution[i], size[i], cudaMemcpyDeviceToHost);
        stbi_write_png(fileNameOutConvolution[i], width[i] - 2, height[i] - 2, 4, imageDataConvolutionHost[i], 4 * width[i]);
    }
    printf("Done\r\n");

    // Writing min pooled images
    printf("Writing min pooling png to disk\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        printf("Image %d of %d\r\n", i + 1, 10);
        cudaMemcpy(imageDataMinPoolingHost[i], imageDataMinPooling[i], size[i], cudaMemcpyDeviceToHost);
        stbi_write_png(fileNameOutMinPooling[i], width[i] / 2, height[i] / 2, 4, imageDataMinPoolingHost[i], 4 * (width[i] / 2));
    }
    printf("Done\r\n");

    // Writing max pooled images
    printf("Writing max pooling png to disk\r\n");
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        printf("Image %d of %d\r\n", i + 1, 10);
        cudaMemcpy(imageDataMaxPoolingHost[i], imageDataMaxPooling[i], size[i], cudaMemcpyDeviceToHost);
        stbi_write_png(fileNameOutMaxPooling[i], width[i] / 2, height[i] / 2, 4, imageDataMaxPoolingHost[i], 4 * (width[i] / 2));
    }
    printf("Done\r\n");

    // Free memory and destroy streams
    for (int i = 0; i < NUMBER_OF_IMAGES; i++)
    {
        stbi_image_free(originalImageHost[i]);

        free(imageDataConvolutionHost[i]);
        free(imageDataMinPoolingHost[i]);
        free(imageDataMaxPoolingHost[i]);

        cudaFree(originalImage[i]);
        cudaFree(imageDataConvolution[i]);
        cudaFree(imageDataMinPooling[i]);
        cudaFree(imageDataMaxPooling[i]);

        cudaStreamDestroy(stream[i]);
    }

    timer_end = clock(); // end the timer
    double time_spent = (double)(timer_end - timer_start) / CLOCKS_PER_SEC;
    printf("\nTotal program time: %.3fs\n", time_spent);

    double time_spent_process = (double)(timer_end_process - timer_start_process) / CLOCKS_PER_SEC;
    printf("\nProcessing time (CUDA Kernels): %.3fs\n", time_spent_process);

    return 0;
}

__global__ void ConvertImageToGrayCpu(unsigned char *originalImage, unsigned char *imageDataGrayscale, int width, int height)
{
    int idx = (threadIdx.x + blockIdx.x * blockDim.x) * 4;
    int gridStride = blockDim.x * gridDim.x;
    int totalPixels = width * height * 4;

    for (int x = idx; x < totalPixels; x += gridStride)
    {
        Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[x];
        Pixel *ptrPixelOriginal = (Pixel *)&originalImage[x];
        unsigned char pixelValue = (unsigned char)(ptrPixelOriginal->r * 0.2126f + ptrPixelOriginal->g * 0.7152f + ptrPixelOriginal->b * 0.0722f);
        ptrPixel->r = pixelValue;
        ptrPixel->g = pixelValue;
        ptrPixel->b = pixelValue;
        ptrPixel->a = 255;
    }
}

__global__ void convolveImage(unsigned char *imageDataGrayscale, unsigned char *imageDataConvolution, int width, int height)
{
    int idx = (threadIdx.x + blockIdx.x * blockDim.x) * 4;
    int gridStride = blockDim.x * gridDim.x;
    int totalPixels = width * height * 4;

    int kernel[3][3] =
        {
            {1, 0, -1},
            {1, 0, -1},
            {1, 0, -1}};

    int pixels[3][3] = {0};
    int finalPixel = 0;

    for (int x = idx; x < totalPixels - 2; x += gridStride)
    {
        for (int i = 0; i <= 2; i++)
        {
            Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[x + i * 4];

            pixels[0][i] = ptrPixel->r * kernel[0][i];
        }

        for (int i = 0; i <= 2; i++)
        {
            Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[x + width * 4 + i * 4];

            pixels[1][i] = ptrPixel->r * kernel[1][i];
        }

        for (int i = 0; i <= 2; i++)
        {
            Pixel *ptrPixel = (Pixel *)&imageDataGrayscale[x + (2 * width * 4) + i * 4];

            pixels[2][i] = ptrPixel->r * kernel[2][i];
        }

        finalPixel = (pixels[0][0] + pixels[0][1] + pixels[0][2] + pixels[1][0] + pixels[1][1] + pixels[1][2] + pixels[2][0] + pixels[2][1] + pixels[2][2]) / 9;

        Pixel *ptrPixel = (Pixel *)&imageDataConvolution[x];
        ptrPixel->r = finalPixel;
        ptrPixel->g = finalPixel;
        ptrPixel->b = finalPixel;
        ptrPixel->a = 255;
    }
}

__global__ void minPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height)
{
    // Calculate the 2D block index
    int blockY = blockIdx.y;
    int blockX = blockIdx.x;

    // Calculate the starting position of the 2x2 block
    int y = blockY * 2;
    int x = blockX * 2;

    // Calculate the index of the current pixel in the 1D arrays
    int indexOut = blockY * width / 2 * 4 + blockX * 4;

    // For each channel, find the minimum value in the 2x2 block
    for (int c = 0; c < 4; c++)
    {
        unsigned char min = 255;
        for (int dy = 0; dy < 2; dy++)
        {
            for (int dx = 0; dx < 2; dx++)
            {
                // Calculate the index of the current pixel in the 1D array
                int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                unsigned char value = originalImage[index];
                min = (value < min) ? value : min;
            }
        }
        // Store the minimum value in the result array
        minPoolingImage[indexOut + c] = min;
    }
}

__global__ void maxPooling(unsigned char *originalImage, unsigned char *minPoolingImage, int width, int height)
{
    // Calculate the 2D block index
    int blockY = blockIdx.y;
    int blockX = blockIdx.x;

    // Calculate the starting position of the 2x2 block
    int y = blockY * 2;
    int x = blockX * 2;

    // Calculate the index of the current pixel in the 1D arrays
    int indexOut = blockY * width / 2 * 4 + blockX * 4;

    // For each channel, find the minimum value in the 2x2 block
    for (int c = 0; c < 4; c++)
    {
        unsigned char min = 0;
        for (int dy = 0; dy < 2; dy++)
        {
            for (int dx = 0; dx < 2; dx++)
            {
                // Calculate the index of the current pixel in the 1D array
                int index = (y + dy) * width * 4 + (x + dx) * 4 + c;
                unsigned char value = originalImage[index];
                min = (value > min) ? value : min;
            }
        }
        // Store the minimum value in the result array
        minPoolingImage[indexOut + c] = min;
    }
}



Writing ImageFilter_CUDA.cu


In [ ]:
!nvcc ImageFilter_CUDA.cu -o ImageFilter_CUDA --generate-line-info

stb_image.h(4207): warning: variable "old_limit" was set but never used

stb_image.h(5121): warning: variable "idata_limit_old" was set but never used

stb_image.h(6894): warning: variable "out_size" was set but never used

stb_image.h(6895): warning: variable "delays_size" was set but never used



In [ ]:
!nvprof --print-gpu-trace ./ImageFilter_CUDA

==1219== NVPROF is profiling process 1219, command: ./ImageFilter_CUDA
Building filepaths
Done
Loading png files
Done
Processing images grayscale
Done
Processing image convolution
Done
Processing images minimum pooling
Done
Processing image maximum pooling
Done
Writing convolved png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done
Writing min pooling png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done
Writing max pooling png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done

Total program time: 8.696s

Processing time (CUDA Kernels): 0.034s
==1219== Profiling application: ./ImageFilter_CUDA
==1219== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
525.97ms  3.4880us                    -               -         -         -         -  16.000KB  4.3747GB/s    Pageable      Device     Tesl

In [ ]:
!nsys profile ImageFilter_CUDA

Building filepaths
Done
Loading png files
Done
Processing images grayscale
Done
Processing image convolution
Done
Processing images minimum pooling
Done
Processing image maximum pooling
Done
Writing convolved png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done
Writing min pooling png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done
Writing max pooling png to disk
Image 1 of 10
Image 2 of 10
Image 3 of 10
Image 4 of 10
Done

Total program time: 9.839s

Processing time (CUDA Kernels): 0.034s
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-faeb-951e-168c-7876.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-faeb-951e-168c-7876.qdrep"
Report file moved to "/content/report1.qdrep"


In [ ]:
!ncu --set full -o ImageFilter_CUDA ./ImageFilter_CUDA

==PROF== Connected to process 1431 (/content/ImageFilter_CUDA)
Building filepaths
Done
Loading png files
Done
Processing images grayscale
==PROF== Profiling "ConvertImageToGrayCpu" - 1: 0%....50%....100% - 32 passes
==PROF== Profiling "ConvertImageToGrayCpu" - 2: 0%....50%....100% - 32 passes
==PROF== Profiling "ConvertImageToGrayCpu" - 3: 0%....50%....100% - 32 passes
==PROF== Profiling "ConvertImageToGrayCpu" - 4: 0%....50%....100% - 32 passes
Done
Processing image convolution
==PROF== Profiling "convolveImage" - 5: 0%....50%....100% - 32 passes
==PROF== Profiling "convolveImage" - 6: 0%....50%....100% - 32 passes
==PROF== Profiling "convolveImage" - 7: 0%....50%....100% - 32 passes
==PROF== Profiling "convolveImage" - 8: 0%....50%....100% - 32 passes
Done
Processing images minimum pooling
==PROF== Profiling "minPooling" - 9: 0%....50%....100% - 32 passes
==PROF== Profiling "minPooling" - 10: 0%....50%....100% - 32 passes
==PROF== Profiling "minPooling" - 11: 0%....50%....100% - 32 p